In [1]:
import glob
import os
import re
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import spacy

In [2]:
test = os.path.join("plain", "36105213330389.txt")

In [3]:
nlp = spacy.load('en_core_web_sm')

In [33]:
def strip_names_and_numbers(text):
    doc = nlp(text)
    tokens = [ent.text for ent in doc.ents if ent.label_ not in ["PERSON", "CARDINAL"]]
    return " ".join(tokens)

In [36]:
with open(test, 'r') as f:
    text = f.read()
    de_personed = strip_names_and_numbers(text)
    print(de_personed)

The White Hat &C. &C. ' v.- ARROWSMITH SERIES BOAT I. Eden Phillpotts THE END OF A LIFE Yol FRENCHMAN SUPERNATURAL THE WHITE HAT Vol Quay Street London Kent & Co. “FLOWERS OF THE hunt Quay Street LONDON Kent & Co. The Vicissitudes of a Hack “Little Fairy” “Got At” the Illustrated, Sporting and Dramatic News THE WHITE HAT HACK LITTLE FAIRY HOUSE GOT THE QUEEN OF THE ARENA THE BLACK SHEEP OF FAMILY GAME a White Hat the moon, one night the following week THE WHITE HAT Cousin the next morning Regent Street THE WHITE HAT ’at first half a minute two minutes Retreat-6 WHITE HAT a couple of minutes Skeffington Downshire Grace WHITE HAT Downshire Downshire next week Duchess Downshire Downshire THE WHITE HAT Downshire Downshire Two days later the “ Goodwood IO Snob Snob Downshire THE WHITE HAT Dukes at least half an hour Two days later some days the Queen’s Hotel about five o’clock the afternoon steps.12 THE WHITE HAT second Downshire THE WHITE HAT nearly eight o’clock Goodwood morning Cousin WH

In [26]:
def lowercase(textlist):
    return [word.lower() for word in textlist]

In [27]:
def remove_numbers(text):
    cleaned = re.sub(r'\d+', '', text)
    return cleaned

In [28]:
def remove_non_alpha_nltk(text):
    words = word_tokenize(text)
    lowered = lowercase(words)
    return " ".join([word for word in lowered if word.isalpha()])

In [29]:
def remove_single_letter_words(text):
    cleaned = ' '.join([word for word in text.split() if len(word) > 1])
    return cleaned

In [31]:
def clean_text(text):
    de_person_number = strip_names_and_numbers(text)
    alpha = remove_non_alpha_nltk(de_person_number)
    cleaned = remove_single_letter_words(alpha)
    return cleaned

In [35]:
with open(test, 'r') as f:
    text = f.read()
    cleaned = clean_text(text)
    print(cleaned)

the white hat arrowsmith series boat eden phillpotts the end of life yol frenchman supernatural the white hat vol quay street london kent flowers of the hunt quay street london kent co the vicissitudes of hack little fairy got at the illustrated sporting and dramatic news the white hat hack little fairy house got the queen of the arena the black sheep of family game white hat the moon one night the following week the white hat cousin the next morning regent street the white hat at first half minute two minutes white hat couple of minutes skeffington downshire grace white hat downshire downshire next week duchess downshire downshire the white hat downshire downshire two days later the goodwood io snob snob downshire the white hat dukes at least half an hour two days later some days the queen hotel about five clock the afternoon the white hat second downshire the white hat nearly eight clock goodwood morning cousin white hat me white hat my salad days first swell years old hack first swe

In [66]:
# Given a filename, extract just the book id
def get_book_id(fn):
    return os.path.split(fn)[1].split('_')[0].split('.')[0]

In [69]:
for fn in tqdm(glob.glob(os.path.join('plain', '*.txt'))):
    book_id = get_book_id(fn)
    with open(fn, 'r') as f:
        with open(os.path.join('tm_texts', book_id + '.txt'), 'w') as g:
            text = f.read()
            g.write(clean_text(text))

100%|██████████| 1417/1417 [15:02<00:00,  1.22it/s]
